In [1]:
import numpy as np

In [2]:
"""Mounting Google Drive"""
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
import os

!ls /content/drive/MyDrive/EEG\ Strokes\ Dataset/

edffile.zip  sourcedata.zip


In [4]:
import os

os.makedirs('/tmp/Strokes Data/', exist_ok=True)
!unzip "/content/drive/MyDrive/EEG Strokes Dataset/sourcedata.zip" -d "/tmp/Strokes Data/"

Archive:  /content/drive/MyDrive/EEG Strokes Dataset/sourcedata.zip
   creating: /tmp/Strokes Data/sourcedata/
   creating: /tmp/Strokes Data/sourcedata/sub-01/
 extracting: /tmp/Strokes Data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-02/
 extracting: /tmp/Strokes Data/sourcedata/sub-02/sub-02_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-03/
 extracting: /tmp/Strokes Data/sourcedata/sub-03/sub-03_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-04/
 extracting: /tmp/Strokes Data/sourcedata/sub-04/sub-04_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-05/
 extracting: /tmp/Strokes Data/sourcedata/sub-05/sub-05_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-06/
 extracting: /tmp/Strokes Data/sourcedata/sub-06/sub-06_task-motor-imagery_eeg.mat  
   creating: /tmp/Strokes Data/sourcedata/sub-07/
 extracting: /tmp/Strokes Dat

In [5]:
import os

mat_file_path = '/tmp/Strokes Data/sourcedata/'

mat_file_paths = []

for root, dirs, files in os.walk(mat_file_path):
    for file in files:
        if file.endswith('.mat'):
            full_path = os.path.join(root, file)
            mat_file_paths.append(full_path)

In [6]:
import scipy.io

eeg_data = []
mi_labels = []
for files in mat_file_paths:
  mat_data = scipy.io.loadmat(files)
  data = mat_data['eeg'][0][0][0]
  labels = mat_data['eeg'][0][0][1]
  labels = labels.flatten()
  eeg_data.append(data)
  mi_labels.append(labels)

In [10]:
eeg_data_reshaped = np.array(eeg_data).reshape(50 * 40, 33, 4000)  # Shape: (2000, 33, 4000)
mi_labels_reshaped = np.array(mi_labels).flatten()  # Shape: (2000,)

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming eeg_data_reshaped and mi_labels_reshaped are already loaded

# Number of subjects
num_subjects = 2000 // 40

# Initialize lists to store mean and std dev for each subject
mean_per_subject = []
std_per_subject = []

# Calculate mean and std deviation for each subject
for i in range(num_subjects):
    start_idx = i * 40
    end_idx = (i + 1) * 40
    subject_data = eeg_data_reshaped[start_idx:end_idx, :, :]

    # Calculate mean and std deviation across all trials for each channel
    mean_subject = np.mean(subject_data, axis=0)  # mean across trials
    std_subject = np.std(subject_data, axis=0)    # std deviation across trials

    # Aggregate mean and std deviation for all channels
    mean_per_subject.append(np.mean(mean_subject, axis=1))
    std_per_subject.append(np.mean(std_subject, axis=1))

# Convert lists to numpy arrays
mean_per_subject = np.array(mean_per_subject)
std_per_subject = np.array(std_per_subject)

# Create a DataFrame for easy plotting and analysis
df_stats = pd.DataFrame({
    'Subject': np.arange(num_subjects),
    'Mean': np.mean(mean_per_subject, axis=1),
    'Std_Dev': np.mean(std_per_subject, axis=1)
})

# Statistical Analysis
anova_mean = stats.f_oneway(*[df_stats['Mean'][i::5] for i in range(5)])
anova_std = stats.f_oneway(*[df_stats['Std_Dev'][i::5] for i in range(5)])

# Print ANOVA results
print("ANOVA Results for Mean EEG Signal Across Subjects:")
print(f"F-Statistic: {anova_mean.statistic}, p-Value: {anova_mean.pvalue}")

print("\nANOVA Results for Standard Deviation of EEG Signal Across Subjects:")
print(f"F-Statistic: {anova_std.statistic}, p-Value: {anova_std.pvalue}")

# Plot and save individual plots
# Plot mean
plt.figure(figsize=(12, 6))
sns.boxplot(x='Subject', y='Mean', data=df_stats)
plt.title('Mean EEG Signal Across Subjects')
plt.xlabel('Subject')
plt.ylabel('Mean Value')
plt.xticks(ticks=np.arange(0, num_subjects, 2), labels=np.arange(1, num_subjects+1, 2))
plt.tight_layout()
plt.savefig('mean_eeg_signal.eps')
plt.savefig('mean_eeg_signal.pdf')
plt.close()

# Plot std deviation
plt.figure(figsize=(12, 6))
sns.boxplot(x='Subject', y='Std_Dev', data=df_stats)
plt.title('Standard Deviation of EEG Signal Across Subjects')
plt.xlabel('Subject')
plt.ylabel('Standard Deviation')
plt.xticks(ticks=np.arange(0, num_subjects, 2), labels=np.arange(1, num_subjects+1, 2))
plt.tight_layout()
plt.savefig('std_dev_eeg_signal.eps')
plt.savefig('std_dev_eeg_signal.pdf')
plt.close()

# Plot violin plots for Mean and Std Dev
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_stats[['Mean', 'Std_Dev']])
plt.title('Violin Plot of EEG Signal Statistics')
plt.ylabel('Value')
plt.tight_layout()
plt.savefig('violin_plot_eeg_signal.eps')
plt.savefig('violin_plot_eeg_signal.pdf')
plt.close()

# Plot distribution of mean values
plt.figure(figsize=(12, 6))
sns.histplot(df_stats['Mean'], bins=30, kde=True)
plt.title('Distribution of Mean EEG Signal Across Subjects')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('distribution_mean_eeg_signal.eps')
plt.savefig('distribution_mean_eeg_signal.pdf')
plt.close()

# Save results to CSV if needed
df_stats.to_csv('subject_statistics.csv', index=False)

ANOVA Results for Mean EEG Signal Across Subjects:
F-Statistic: 0.7043602108612822, p-Value: 0.5931248441673141

ANOVA Results for Standard Deviation of EEG Signal Across Subjects:
F-Statistic: 0.4347207652014916, p-Value: 0.7827997433578474


In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 81.1 MB/s eta 0:00:00


In [ ]:
!pip install mne_connectivity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import mne  # For EEG processing and feature extraction
from scipy.signal import hilbert, coherence, welch
from tqdm import tqdm

# Reshape EEG data
eeg_data_reshaped = np.array(eeg_data).reshape(50 * 40, 33, 4000)  # Shape: (2000, 33, 4000)
mi_labels_reshaped = np.array(mi_labels).flatten()  # Shape: (2000,)

# Parameters
window_size = 1000
step_size = 500  # Overlap step size
fs = 500

# Calculate the number of windows
num_samples = eeg_data_reshaped.shape[0]
num_channels = eeg_data_reshaped.shape[1]
num_windows = (eeg_data_reshaped.shape[2] - window_size) // step_size + 1

# Initialize feature matrix
features_matrix = np.zeros((num_samples, num_channels, num_windows, 8))  # 8 features per window

# Functions for feature calculations
def calculate_band_power(signal, fs, band):
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2)
    band_idx = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.trapz(psd[band_idx], freqs[band_idx])

def hilbert_huang_transform(signal):
    analytic_signal = hilbert(signal)
    amplitude_envelope = np.abs(analytic_signal)
    return np.mean(amplitude_envelope), np.std(amplitude_envelope)

def calculate_coherence(signal1, signal2, fs):
    f, Cxy = coherence(signal1, signal2, fs=fs, nperseg=fs*2)
    return np.mean(Cxy)

def calculate_erd_ers(signal, fs, motor_band):
    return calculate_band_power(signal, fs, motor_band)  # Placeholder for ERD/ERS calculation

def fractal_dimension(signal):
    return np.log(np.std(signal) / np.mean(signal))  # Placeholder calculation

def lyapunov_exponent(signal):
    return np.mean(np.diff(signal))  # Placeholder calculation

# Feature extraction process
for sample_idx in tqdm(range(num_samples), desc='Processing Samples'):
    for channel_idx in range(num_channels):
        channel_data = eeg_data_reshaped[sample_idx, channel_idx, :]

        for window_idx, start in enumerate(range(0, channel_data.shape[0] - window_size + 1, step_size)):
            window_data = channel_data[start:start + window_size]

            # Feature calculations for each window
            alpha_power = calculate_band_power(window_data, fs, band=(8, 13))
            beta_power = calculate_band_power(window_data, fs, band=(13, 30))
            mean_envelope, std_envelope = hilbert_huang_transform(window_data)
            fractal_dim = fractal_dimension(window_data)
            lyapunov_exp = lyapunov_exponent(window_data)

            if channel_idx < num_channels - 1:
                coherence_val = calculate_coherence(window_data, eeg_data_reshaped[sample_idx, channel_idx + 1, start:start + window_size], fs)
            else:
                coherence_val = calculate_coherence(window_data, window_data, fs)  # Self-coherence as fallback

            erd_ers_val = calculate_erd_ers(window_data, fs, motor_band=(8, 13))

            # Store the features in the features_matrix
            features_matrix[sample_idx, channel_idx, window_idx, :] = [
                alpha_power, beta_power, mean_envelope, std_envelope,
                coherence_val, erd_ers_val, fractal_dim, lyapunov_exp
            ]

# Check the output shape
print("Features matrix shape:", features_matrix.shape)

Processing Samples:   0%|          | 0/2000 [00:00<?, ?it/s]<ipython-input-22-da846db07fab>:42: RuntimeWarning: invalid value encountered in log
  return np.log(np.std(signal) / np.mean(signal))  # Placeholder calculation
<ipython-input-22-da846db07fab>:42: RuntimeWarning: divide by zero encountered in log
  return np.log(np.std(signal) / np.mean(signal))  # Placeholder calculation
<ipython-input-22-da846db07fab>:42: RuntimeWarning: invalid value encountered in scalar divide
  return np.log(np.std(signal) / np.mean(signal))  # Placeholder calculation
Processing Samples: 100%|██████████| 2000/2000 [18:38<00:00,  1.79it/s]

Features matrix shape: (2000, 33, 7, 8)


In [ ]:
features_matrix

array([[[[ 4.65331303e+00,  4.19538473e+01,  8.19505719e+03, ...,
           4.65331303e+00, -4.63508384e+00,  2.20049456e-01],
         [ 9.92111954e+00,  1.27062865e+01,  8.27872102e+03, ...,
           9.92111954e+00, -5.24937190e+00,  7.92715021e-02],
         [ 5.50293946e+00,  8.18777170e+00,  8.32200924e+03, ...,
           5.50293946e+00, -5.96434054e+00,  8.84001425e-02],
         ...,
         [ 8.55333963e+00,  1.02387059e+01,  8.31580803e+03, ...,
           8.55333963e+00, -4.88097240e+00, -2.35151986e-02],
         [ 4.02404377e+00,  7.49470654e+00,  8.29405707e+03, ...,
           4.02404377e+00, -4.58012904e+00, -1.51608158e-01],
         [ 5.01491607e+00,  1.23665618e+01,  8.16667780e+03, ...,
           5.01491607e+00, -4.67958525e+00, -2.42058212e-01]],

        [[ 4.43968498e+00,  1.74736801e+01,  1.23324091e+04, ...,
           4.43968498e+00, -5.11089714e+00,  2.05461531e-01],
         [ 7.12432667e+00,  1.05021124e+01,  1.23988511e+04, ...,
           7.12432667e

In [ ]:
import pickle

filename = '/content/drive/MyDrive/EEG_Storkes_Features_Journal.pkl'

with open(filename, 'wb') as file:
    pickle.dump({'features_matrix': features_matrix, 'mi_labels': mi_labels}, file)

print(f"Data saved to {filename}")

Data saved to /content/drive/MyDrive/EEG_Storkes_Features_Journal.pkl
